# Linear Regression

**Imports**

In [54]:
import findspark
findspark.init('/home/sedat/spark-3.3.2-bin-hadoop3')
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

**Start session and read csv file**

In [3]:
spark = SparkSession.builder.appName('customers').getOrCreate()

In [9]:
all_data = spark.read.csv('Ecommerce_Customers.csv', inferSchema=True, header=True)

**Data Types**

In [14]:
all_data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



**Total observation units**

In [10]:
all_data.count()

500

**Column names**

In [11]:
all_data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

**First observation unit**

In [12]:
all_data.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005)]

In [30]:
for key,value in enumerate(all_data.head(1)[0]):
    print(all_data.columns[key] + ': ', value)

Email:  mstephenson@fernandez.com
Address:  835 Frank TunnelWrightmouth, MI 82180-9605
Avatar:  Violet
Avg Session Length:  34.49726772511229
Time on App:  12.65565114916675
Time on Website:  39.57766801952616
Length of Membership:  4.0826206329529615
Yearly Amount Spent:  587.9510539684005


**Vector Assembler (Assign Features and Label)**

In [35]:
all_data.columns[3:-1]

['Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership']

In [36]:
assembler = VectorAssembler(inputCols=all_data.columns[3:-1], outputCol='features')

In [38]:
output = assembler.transform(all_data)

In [41]:
output.head(1)[0]

Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))

In [42]:
final_data = output.select('features', 'Yearly Amount Spent')

In [43]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

**Linear Regression Model**

In [45]:
# split data to train and test
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [48]:
print(f'Train data: {train_data.count()}')
print(f'Test data: {test_data.count()}')

Train data: 349
Test data: 151


In [49]:
# describe train data
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                349|
|   mean| 494.53962380143287|
| stddev|  81.81934448390658|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [51]:
# describe test data
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                151|
|   mean|  510.3489431959953|
| stddev|  72.25560466728332|
|    min| 347.77692663187264|
|    max|  712.3963268096637|
+-------+-------------------+



In [55]:
# model
lr = LinearRegression(featuresCol='features', labelCol='Yearly Amount Spent', predictionCol='predictions')
lrModel = lr.fit(train_data)

23/03/19 15:09:00 WARN Instrumentation: [6adbe070] regParam is zero, which might cause numerical instability and overfitting.


In [60]:
# check test residuals
test_results = lrModel.evaluate(test_data)
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|-3.4537816008552795|
|-12.792657717534155|
|-21.566750113538887|
| 21.860549234884274|
|  4.243601495695998|
|-3.7633729866720387|
| 3.3231265030223653|
|  4.650158282601126|
|-17.658587850005915|
|-14.361817923571152|
|   17.6480654187215|
|  7.238000734832156|
|-0.5705018994812008|
|-0.9245696148645379|
| -5.931454301247584|
| -4.330133215024546|
|  7.808039849875229|
|  8.562380069164021|
| 6.0448343459464695|
|  8.544091272195715|
+-------------------+
only showing top 20 rows



In [62]:
# root mean squared error
test_results.rootMeanSquaredError

10.563875149114937

In [63]:
# r2
test_results.r2

0.978482632948155